In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv(r"C:\Users\asus\Downloads\spaceship-titanic\train.csv")
test_data = pd.read_csv(r"C:\Users\asus\Downloads\spaceship-titanic\test.csv")
submission = pd.read_csv(r"C:\Users\asus\Downloads\spaceship-titanic\sample_submission.csv")
df = pd.concat([train_data, test_data])

In [3]:
df.tail()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,NaN
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,NaN
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,NaN
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,NaN
4276,9277_01,Earth,True,G/1498/S,PSO J318.5-22,43.0,False,0.0,0.0,0.0,0.0,0.0,Lilace Leonzaley,NaN


In [4]:
train_data.shape

(8693, 14)

* Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.


In [5]:
transported = train_data["Transported"]

In [6]:
new = df["Cabin"].str.split("/", n = 2, expand = True)

In [7]:
new

,0,1,2
0,B,0,P
1,F,0,S
2,A,0,S
3,A,0,S
4,F,1,S
...,...,...,...
4272,G,1496,S
4273,NaN,NaN,NaN
4274,D,296,P
4275,D,297,P


In [8]:
df["deck"] = new[0]
df["num"] = new[1]
df["side"] = new[2]

In [9]:
id = df["PassengerId"].str.split("_", n = 1, expand = True)
df["group"] = id[0]

In [10]:

df = df.drop(columns = ["PassengerId", "Cabin", "Name", "Transported"], axis = 1)

In [11]:
df.head(30)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,group
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0,P,0001
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0,S,0002
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S,0003
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S,0003
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1,S,0004
5,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,F,0,P,0005
6,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,F,2,S,0006
7,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,G,0,S,0006
8,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,F,3,S,0007
9,Europa,True,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,B,1,P,0008


In [12]:
df.shape


(12970, 14)

In [13]:
df.isnull().sum()

HomePlanet      288
CryoSleep       310
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
deck            299
num             299
side            299
group             0
dtype: int64

In [14]:
col_list = df.columns
col_list

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'deck', 'num', 'side',
       'group'],
      dtype='object')

In [15]:
for col in col_list:
    if (df[col].dtypes == "object"):
        df[col] = df[col].fillna(df[col].mode()[0])
        
    else:
        df[col] = df[col].fillna(df[col].mean())

In [16]:
df.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
deck            0
num             0
side            0
group           0
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12970 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12970 non-null  object 
 1   CryoSleep     12970 non-null  bool   
 2   Destination   12970 non-null  object 
 3   Age           12970 non-null  float64
 4   VIP           12970 non-null  bool   
 5   RoomService   12970 non-null  float64
 6   FoodCourt     12970 non-null  float64
 7   ShoppingMall  12970 non-null  float64
 8   Spa           12970 non-null  float64
 9   VRDeck        12970 non-null  float64
 10  deck          12970 non-null  object 
 11  num           12970 non-null  object 
 12  side          12970 non-null  object 
 13  group         12970 non-null  object 
dtypes: bool(2), float64(6), object(6)
memory usage: 1.3+ MB


In [18]:
df = df.astype({'num':'float','group':'float'})

In [19]:
cat_col = []
num_col = []
for col in col_list:
    if (df[col].dtype == "object") | (df[col].dtype == "bool"):
        cat_col.append(col)
    else:
        num_col.append(col)
        
df[cat_col]

,HomePlanet,CryoSleep,Destination,VIP,deck,side
0,Europa,False,TRAPPIST-1e,False,B,P
1,Earth,False,TRAPPIST-1e,False,F,S
2,Europa,False,TRAPPIST-1e,True,A,S
3,Europa,False,TRAPPIST-1e,False,A,S
4,Earth,False,TRAPPIST-1e,False,F,S
...,...,...,...,...,...,...
4272,Earth,True,TRAPPIST-1e,False,G,S
4273,Earth,False,TRAPPIST-1e,False,F,S
4274,Mars,True,55 Cancri e,False,D,P
4275,Europa,False,TRAPPIST-1e,False,D,P


In [20]:
num_col

['Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'num',
 'group']

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12970 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12970 non-null  object 
 1   CryoSleep     12970 non-null  bool   
 2   Destination   12970 non-null  object 
 3   Age           12970 non-null  float64
 4   VIP           12970 non-null  bool   
 5   RoomService   12970 non-null  float64
 6   FoodCourt     12970 non-null  float64
 7   ShoppingMall  12970 non-null  float64
 8   Spa           12970 non-null  float64
 9   VRDeck        12970 non-null  float64
 10  deck          12970 non-null  object 
 11  num           12970 non-null  float64
 12  side          12970 non-null  object 
 13  group         12970 non-null  float64
dtypes: bool(2), float64(8), object(4)
memory usage: 1.3+ MB


In [22]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
encoder = LabelEncoder()
for col in cat_col:
    df[col] = encoder.fit_transform(df[col])

In [23]:
scaler = StandardScaler()
scaler.fit(df[num_col])
df[num_col] = scaler.transform(df[num_col])

In [ ]:
X

In [24]:
train_preprocessed = df[0:8693]
test_preprocessed = df[8693:]
X = train_preprocessed
y = transported
feature_columns = train_preprocessed.columns
y = y.replace({False : 0, True : 1})

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=0,
  
)

In [40]:
test_preprocessed[feature_columns]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,group
0,0,1,2,-1.244695e-01,0,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,6,-1.147473,1,-1.721028
1,0,0,2,-6.864188e-01,0,-0.347750,-0.282761,-0.299739,2.249544,-0.262708,5,-1.145523,1,-1.719166
2,1,1,0,1.565051e-01,0,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,2,-1.153321,1,-1.718794
3,1,0,2,6.482107e-01,0,-0.347750,3.957746,-0.299739,-0.114043,0.238236,2,-1.151372,1,-1.718049
4,0,0,2,-6.161751e-01,0,-0.332148,-0.288506,0.788470,-0.275970,-0.262708,5,-1.143573,1,-1.717304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0,1,2,3.672360e-01,0,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,6,1.763235,1,1.724127
4273,0,0,2,9.291853e-01,0,-0.347750,0.252170,-0.270606,-0.267024,-0.139399,5,-0.993457,1,1.725244
4274,2,1,0,-2.495556e-16,0,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,3,-0.576249,0,1.725989
4275,1,0,2,-2.495556e-16,0,-0.347750,1.422251,-0.299739,-0.275970,0.185144,3,-0.574299,0,1.726734


In [42]:
from sklearn.metrics import *


0.7825920245398773

In [45]:
from sklearn.ensemble import RandomForestClassifier


In [46]:
rfc = RandomForestClassifier(max_depth=15, n_estimators=200, random_state=0)
rfc.fit(X_train, y_train)
rfc_test_pred = rfc.predict(X_test)
acc = accuracy_score(y_test, rfc_test_pred)
acc

0.8101993865030674

In [47]:
# https://www.kaggle.com/code/vineetkumarpatil/spaceship-titanic-using-logisticregression

In [67]:
model_predictions = rfc.predict(test_preprocessed[feature_columns])
results = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Transported': model_predictions})
results.Transported = results.Transported.replace({0: False, 1: True})
results.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [68]:
results.to_csv('submission.csv', index=False)